In [1]:
from docx import Document
import re
import time
import random
import json
import os
import zipfile
import shutil
from lxml import etree
from docx.oxml.shared import OxmlElement
from docx.oxml import ns
from docx.oxml.ns import qn
import pickle

In [2]:
input_path = "input/input.docx"
output_path = "output.docx"
temp_dir = "docx_temp"

W_NS = "http://schemas.openxmlformats.org/wordprocessingml/2006/main"
nsmap = {"w": W_NS} 

In [3]:
with open('sentences.pkl', 'rb') as f:
    sentences = pickle.load(f)

In [4]:
print(sentences[:5])

['(A) Potential unethical:', "Fred creates an ethical conflict by joining Phaust, risking misuse of Chemitoil's information from his earlier work.", "Phaust gives importance to profit over environmental protection, exploiting Mexico's less stringent regulations like no lining on ponds, causing toxic wastewater to harm resources and communities.", '(B) Technical Problems:', 'The lack of safety testing on equipment from a new vendor caused catastrophic results.']


In [5]:
def unzip_docx(input_docx, extract_to):
    if os.path.exists(extract_to):
        shutil.rmtree(extract_to)
    os.makedirs(extract_to)

    with zipfile.ZipFile(input_docx, 'r') as zip_ref:
        zip_ref.extractall(extract_to)


In [6]:
def zip_to_docx(folder_path, output_docx):
    with zipfile.ZipFile(output_docx, 'w', zipfile.ZIP_DEFLATED) as docx_file:
        for root, _, files in os.walk(folder_path):
            for file in files:
                filepath = os.path.join(root, file)
                arcname = os.path.relpath(filepath, folder_path)
                docx_file.write(filepath, arcname)


In [7]:
def comment_sentences_by_split_run(input_docx, output_docx, sentences):
    temp_dir = "docx_temp"
    unzip_docx(input_docx, temp_dir)

    doc_path = os.path.join(temp_dir, "word", "document.xml")
    doc_tree = etree.parse(doc_path)
    doc_root = doc_tree.getroot()

    comments_path = os.path.join(temp_dir, "word", "comments.xml")
    if os.path.exists(comments_path):
        comments_tree = etree.parse(comments_path)
        comments_root = comments_tree.getroot()
    else:
        comments_root = etree.Element(f"{{{W_NS}}}comments", nsmap=nsmap)
        comments_tree = etree.ElementTree(comments_root)

    paragraphs = doc_root.findall(".//w:body/w:p", namespaces=nsmap)
    comment_id = 0

    for para in paragraphs:
        runs = para.findall(".//w:r", namespaces=nsmap)   
        run_texts = []
        for run in runs:
            t_elem = run.find(".//w:t", namespaces=nsmap)
            run_texts.append(t_elem.text if t_elem is not None and t_elem.text else "")

        full_text = ''.join(run_texts)
        
        if not full_text.strip():
            continue

        for sentence in sentences:
            start_idx = full_text.find(sentence)
            if start_idx == -1:
                continue

            end_idx = start_idx + len(sentence)
            curr_pos = 0
            matched_runs = []

            for run in runs:
                t_elem = run.find(".//w:t", namespaces=nsmap)
                if t_elem is None or t_elem.text is None:
                    i += 1
                    continue
            
                text = t_elem.text
                print(text)
                
            # next_pos = curr_pos + len(text)
            
            #     if next_pos <= start_idx:
            #         curr_pos = next_pos
            #         i += 1
            #         continue
            #     elif curr_pos >= end_idx:
            #         break
            #     else:
            #         rel_start = max(0, start_idx - curr_pos)
            #         rel_end = min(len(text), end_idx - curr_pos)
            
            #         if rel_start > 0 or rel_end < len(text):
            #             before = text[:rel_start]
            #             match = text[rel_start:rel_end]
            #             after = text[rel_end:]
            
            #             if match:
            #                 match_run = etree.fromstring(etree.tostring(run))
            #                 match_run.find(".//w:t", namespaces=nsmap).text = match
            #                 matched_runs.append(match_run)
            #         else:
            #             matched_runs.append(run)
            
            #     curr_pos = next_pos
            #     i += 1
            
            # first_run = matched_runs[0]
            # last_run = matched_runs[-1]

            # comment_start = etree.Element(f"{{{W_NS}}}commentRangeStart", {f"{{{W_NS}}}id": str(comment_id)})
            # comment_end = etree.Element(f"{{{W_NS}}}commentRangeEnd", {f"{{{W_NS}}}id": str(comment_id)})
            # comment_ref_run = etree.Element(f"{{{W_NS}}}r")
            # etree.SubElement(comment_ref_run, f"{{{W_NS}}}commentReference", {f"{{{W_NS}}}id": str(comment_id)})


            # comment = etree.SubElement(comments_root, f"{{{W_NS}}}comment", {
            #     f"{{{W_NS}}}id": str(comment_id),
            #     f"{{{W_NS}}}author": "GPT",
            #     f"{{{W_NS}}}date": "2025-06-04T00:00:00Z"
            # })
            # p = etree.SubElement(comment, f"{{{W_NS}}}p")
            # r = etree.SubElement(p, f"{{{W_NS}}}r")
            # t = etree.SubElement(r, f"{{{W_NS}}}t")
            # t.text = f"{sentence}"

            comment_id += 1

    # # Save updated XMLs
    # doc_tree.write(doc_path, xml_declaration=True, encoding="utf-8", standalone="yes")
    # comments_tree.write(comments_path, xml_declaration=True, encoding="utf-8", standalone="yes")

    # # Add relationship to comments.xml
    # rels_path = os.path.join(temp_dir, "word", "_rels", "document.xml.rels")
    # rels_tree = etree.parse(rels_path)
    # rels_root = rels_tree.getroot()
    # if not any(rel.attrib.get("Target") == "comments.xml" for rel in rels_root.findall("Relationship")):
    #     etree.SubElement(rels_root, "Relationship", {
    #         "Id": "rId100",
    #         "Type": "http://schemas.openxmlformats.org/officeDocument/2006/relationships/comments",
    #         "Target": "comments.xml"
    #     })
    #     rels_tree.write(rels_path, xml_declaration=True, encoding="utf-8", standalone="yes")

    # # Add [Content_Types] override
    # ct_path = os.path.join(temp_dir, "[Content_Types].xml")
    # ct_tree = etree.parse(ct_path)
    # ct_root = ct_tree.getroot()
    # if not any(el.attrib.get("PartName") == "/word/comments.xml" for el in ct_root.findall("Override")):
    #     etree.SubElement(ct_root, "Override", {
    #         "PartName": "/word/comments.xml",
    #         "ContentType": "application/vnd.openxmlformats-officedocument.wordprocessingml.comments+xml"
    #     })
    #     ct_tree.write(ct_path, xml_declaration=True, encoding="utf-8", standalone="yes")

    # zip_to_docx(temp_dir, output_docx)
    # shutil.rmtree(temp_dir)
    # print(f"✅ Comments applied and saved: {output_docx}")


In [8]:
comment_sentences_by_split_run(input_path, output_path, sentences)

(A) 
Potential unethical:
Fred creates an ethical conflict by joining Phaust, risking misuse of Chemitoil's information from his 
earlier
 work.
 
Phaust 
gives importance to
 profit over environmental protection, exploiting Mexico's 
less stringent
 regulations
 like no lining on ponds
, causing toxic 
waste
water
 to harm resources and communities.
Fred creates an ethical conflict by joining Phaust, risking misuse of Chemitoil's information from his 
earlier
 work.
 
Phaust 
gives importance to
 profit over environmental protection, exploiting Mexico's 
less stringent
 regulations
 like no lining on ponds
, causing toxic 
waste
water
 to harm resources and communities.
(B) 
Technical Problems:
The 
lack of safety testing on equipment from a 
new 
vendor caused 
catastrophic results
. 
Using substandard controllers to cut costs due to 
the reduced
 budget 
(20% cut) 
caused technical problems, as the controller software had defects.
The 
lack of safety testing on equipment from a 
new

In [22]:
def comment_sentences_by_split_run(input_docx, output_docx, sentences):
    temp_dir = "docx_temp"
    unzip_docx(input_docx, temp_dir)

    doc_path = os.path.join(temp_dir, "word", "document.xml")
    doc_tree = etree.parse(doc_path)
    doc_root = doc_tree.getroot()

    # === Prepare comments.xml ===
    comments_path = os.path.join(temp_dir, "word", "comments.xml")
    if os.path.exists(comments_path):
        comments_tree = etree.parse(comments_path)
        comments_root = comments_tree.getroot()
    else:
        comments_root = etree.Element(f"{{{W_NS}}}comments", nsmap=nsmap)
        comments_tree = etree.ElementTree(comments_root)

    paragraphs = doc_root.findall(".//w:body/w:p", namespaces=nsmap)
    comment_id = 0

    for para in paragraphs:
        runs = para.findall(".//w:r", namespaces=nsmap)
        run_texts = []
        for run in runs:
            t_elem = run.find(".//w:t", namespaces=nsmap)
            run_texts.append(t_elem.text if t_elem is not None and t_elem.text else "")

        full_text = ''.join(run_texts)
        print(f"\n📝 Full Paragraph Text: {full_text}")
        print(f"📚 Runs in paragraph: {run_texts}")

        for sentence in sentences:
            # Normalize both texts
            normalized_full = ' '.join(full_text.split())
            normalized_sentence = ' '.join(sentence.split())
            
            match = re.search(re.escape(normalized_sentence), normalized_full)
            if not match:
                continue
            
            start_idx = full_text.find(match.group())  # original text index
            end_idx = start_idx + len(match.group())

            print(f"\n🔍 Matching Sentence: {sentence}")
            print(f"   → Start Index: {start_idx}, End Index: {end_idx}")

            curr_pos = 0
            matched_runs = []

            i = 0
            while i < len(runs):
                run = runs[i]
                t_elem = run.find(".//w:t", namespaces=nsmap)
                if t_elem is None or t_elem.text is None:
                    i += 1
                    continue

                run_text = t_elem.text
                next_pos = curr_pos + len(run_text)

                # Check if run overlaps with sentence span
                if next_pos <= start_idx:
                    curr_pos = next_pos
                    i += 1
                    continue
                elif curr_pos >= end_idx:
                    break
                else:
                    rel_start = max(0, start_idx - curr_pos)
                    rel_end = min(len(run_text), end_idx - curr_pos)

                    # Split run at start or end if necessary
                    if rel_start > 0 and rel_end < len(run_text):
                        # Split in two steps: first at rel_end, then rel_start
                        split_run = etree.fromstring(etree.tostring(run))
                        t_split = split_run.find(".//w:t", namespaces=nsmap)

                        # Create three pieces: before, matched, after
                        before = run_text[:rel_start]
                        match = run_text[rel_start:rel_end]
                        after = run_text[rel_end:]

                        # Build runs
                        if before:
                            before_run = etree.fromstring(etree.tostring(run))
                            before_run.find(".//w:t", namespaces=nsmap).text = before
                            run.addprevious(before_run)

                        if match:
                            match_run = etree.fromstring(etree.tostring(run))
                            match_run.find(".//w:t", namespaces=nsmap).text = match
                            run.addprevious(match_run)
                            matched_runs.append(match_run)

                        if after:
                            after_run = etree.fromstring(etree.tostring(run))
                            after_run.find(".//w:t", namespaces=nsmap).text = after
                            run.addprevious(after_run)

                        run.getparent().remove(run)
                        i += 1
                        continue

                    elif rel_start > 0:  # split at start only
                        before = run_text[:rel_start]
                        match = run_text[rel_start:]

                        if before:
                            before_run = etree.fromstring(etree.tostring(run))
                            before_run.find(".//w:t", namespaces=nsmap).text = before
                            run.addprevious(before_run)

                        if match:
                            run.find(".//w:t", namespaces=nsmap).text = match
                            matched_runs.append(run)

                        i += 1
                        curr_pos = next_pos
                        continue

                    elif rel_end < len(run_text):  # split at end only
                        match = run_text[:rel_end]
                        after = run_text[rel_end:]

                        if match:
                            run.find(".//w:t", namespaces=nsmap).text = match
                            matched_runs.append(run)

                        if after:
                            after_run = etree.fromstring(etree.tostring(run))
                            after_run.find(".//w:t", namespaces=nsmap).text = after
                            run.addnext(after_run)

                        i += 1
                        curr_pos = next_pos
                        continue

                    else:
                        matched_runs.append(run)

                curr_pos = next_pos
                i += 1

            if not matched_runs:
                continue

            first_run = matched_runs[0]
            last_run = matched_runs[-1]

            comment_start = etree.Element(f"{{{W_NS}}}commentRangeStart", {f"{{{W_NS}}}id": str(comment_id)})
            comment_end = etree.Element(f"{{{W_NS}}}commentRangeEnd", {f"{{{W_NS}}}id": str(comment_id)})
            comment_ref_run = etree.Element(f"{{{W_NS}}}r")
            etree.SubElement(comment_ref_run, f"{{{W_NS}}}commentReference", {f"{{{W_NS}}}id": str(comment_id)})

            first_run.addprevious(comment_start)
            last_run.addnext(comment_end)
            comment_end.addnext(comment_ref_run)

            comment = etree.SubElement(comments_root, f"{{{W_NS}}}comment", {
                f"{{{W_NS}}}id": str(comment_id),
                f"{{{W_NS}}}author": "GPT",
                f"{{{W_NS}}}date": "2025-06-04T00:00:00Z"
            })
            p = etree.SubElement(comment, f"{{{W_NS}}}p")
            r = etree.SubElement(p, f"{{{W_NS}}}r")
            t = etree.SubElement(r, f"{{{W_NS}}}t")
            t.text = f"Auto comment for: {sentence[:40]}..."

            comment_id += 1

    # Save files
    doc_tree.write(doc_path, xml_declaration=True, encoding="utf-8", standalone="yes")
    comments_tree.write(comments_path, xml_declaration=True, encoding="utf-8", standalone="yes")

    # Add comments relationship
    rels_path = os.path.join(temp_dir, "word", "_rels", "document.xml.rels")
    rels_tree = etree.parse(rels_path)
    rels_root = rels_tree.getroot()
    if not any(rel.attrib.get("Target") == "comments.xml" for rel in rels_root.findall("Relationship")):
        etree.SubElement(rels_root, "Relationship", {
            "Id": "rId100",
            "Type": "http://schemas.openxmlformats.org/officeDocument/2006/relationships/comments",
            "Target": "comments.xml"
        })
        rels_tree.write(rels_path, xml_declaration=True, encoding="utf-8", standalone="yes")

    # Add override in [Content_Types].xml
    ct_path = os.path.join(temp_dir, "[Content_Types].xml")
    ct_tree = etree.parse(ct_path)
    ct_root = ct_tree.getroot()
    if not any(el.attrib.get("PartName") == "/word/comments.xml" for el in ct_root.findall("Override")):
        etree.SubElement(ct_root, "Override", {
            "PartName": "/word/comments.xml",
            "ContentType": "application/vnd.openxmlformats-officedocument.wordprocessingml.comments+xml"
        })
        ct_tree.write(ct_path, xml_declaration=True, encoding="utf-8", standalone="yes")

    # Repack and cleanup
    zip_to_docx(temp_dir, output_docx)
    shutil.rmtree(temp_dir)
    print(f"✅ Comments applied and saved: {output_docx}")


comment_sentences_by_split_run(input_path, output_path, sentences)


📝 Full Paragraph Text: MET – 527 Writing Assignment 4
📚 Runs in paragraph: ['MET – 527 Writing Assignment 4']

📝 Full Paragraph Text: Incident at Morales
📚 Runs in paragraph: ['Incident at Morales']

📝 Full Paragraph Text: Technology from Global Perspective
📚 Runs in paragraph: ['Technology from Global Perspective']

📝 Full Paragraph Text: 
📚 Runs in paragraph: []

📝 Full Paragraph Text: Name: Umashankar Saravanakumar
📚 Runs in paragraph: ['Name: Umashankar Saravanakumar']

📝 Full Paragraph Text: 09/29/2024
📚 Runs in paragraph: ['09/29/2024']

📝 Full Paragraph Text: 
📚 Runs in paragraph: ['']

📝 Full Paragraph Text: Identify two each; (A) potential unethical, (B) technical, and (C) economic issues and problems presented in the Incident at Morales (IaM) story.
📚 Runs in paragraph: ['Identify', ' two each; (A) potential unethical, (B) technical, and (C) economic issues and problems presented in the Incident at Morales (IaM) story.']

📝 Full Paragraph Text: (A) Potential unethical:
📚 Run